In [ ]:
import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.models import Video, UserPreferences
from backend.rating_fields import VIDEO_FIELDS
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, IFrame, display
from IPython.display import YouTubeVideo

To use, enable interactive widgets: https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6

## Please select your preferences

In [ ]:
preferences = {f: widgets.IntSlider(min=-50, max=50, step=1, value=0) for f in VIDEO_FIELDS}

def g(**kwargs):
    global preferences
    for k, v in kwargs.items():
        if k in preferences:
            preferences[k] = v

interact(g, **preferences);

## Please select your experts

In [ ]:
# Selecting experts

usernames = sorted([user.username for user in UserPreferences.objects.all()])

selected = {u: False for u in usernames}
selected['lenhoang'] = True

def f(**kwargs):
    global selected
    for k, v in kwargs.items():
        if k in selected:
            selected[k] = v

interact(f, **selected);

## Select your filters

In [ ]:
videos = Video.objects.all()
videos = [x for x in videos if x.language == 'en']

## Showing sorted videos

In [ ]:
def aggregate_ratings(ratings):
    """Aggregate ratings in a list."""
    by_feature = {f: [getattr(r, f) for r in ratings] for f in VIDEO_FIELDS}
    return {f: np.median(r) if r else np.nan for f, r in by_feature.items()}

def get_score(ratings, preferences):
    """Get score for a video based on preferences and ratings."""
    aggregated = aggregate_ratings(ratings)
    score = np.sum([aggregated[f] * preferences[f] for f in VIDEO_FIELDS])
    return score

def get_score_for_video(video, selected, preferences):
    """Get score for a video based on selected experts and preferences."""
    ratings = [x for x in VideoRating.objects.filter(video=video) if selected[x.user.username]]
    score = get_score(ratings, preferences)
    return score

In [ ]:
# sorting videos
scores = {v: get_score_for_video(v, selected, preferences) for v in videos}
videos_sorted = [x for x in videos if not np.isnan(scores[x])]
videos_sorted = sorted(videos_sorted, key=lambda x: -scores[x])

for v in videos_sorted[:5]:
    print(f"{v.uploader}: {v.name}")
    print(f"Score {round(scores[v], 2)}")
    print(f"Duration {v.duration} / Language {v.language} / Date {v.publication_date} / Views {v.views}")
    
    display(YouTubeVideo(v.video_id))

    print("")